In [1]:
%matplotlib notebook
import firedrake
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np
from constants import *
from hs_solver import *
from phi_solver import *
from model import *

# Introduction
This is a python notebook that details experiments with a 1D-subglacial hydrology model meant to represent the co-evolution of a connected channel-sheet water system based on the Glacier Drainage System model, GlaDS. This model represents the work of many previous studies including the work of Ian Hewitt, Mauro Werder, and Christian Schoof.
## Motivation
Subglacial hydrology beneath ice sheets is very difficult to observe and characterize directly, partially because the water system is very thin relative to the overlying column of kilometer thick ice. For this reason, most of what we know about subglacial hydrology at the ice sheet system scale comes from discharge observations and effective pressure measurements made beneath accessible moutain glaciers. The sampling-rate and data quality of ice sheet observing satallites now make it possible to measure large scale changes in subglacial hydrology manifest at the surface as changes in ice sheet elevation and horizontal velocity.

In Greenland, seasonal surface melt water has been observed to enhance surface velocities near the ice sheet margins by as much as $10-100$ m/yr. The importance of subglacial hydrology for understanding and predicting the evolution of ice sheets in a warming world is still widely debated and depends on the temporal/spatial resolution of the experiment and targeted goals of the study.

The simipliest kind of hydrology model requires three ingredients: 1) Description of water conservation. 2) Description of the hydraulic potential. 3) Time evolution of the drainage space (channel cross section, or sheet thickness). 

## Equations for water conservation
We assume in our model that water is incompressible and that the flux of water through any single node is equal to the flux out of that node plus local sources associated with local melting.

$\frac{dV}{dt} = Q_{in} - Q_{out} + M \Delta s$

## Hydraulic potential

The hydraulic potential beneath an ice sheet can be expressed as the sum of the pressure potential and the elevation potential:

$\phi = p_w + \rho_w g H$

where $H$ is the thickness of the water film.

## Time evolution of conduits (R-channels) and cavities (distributed sheets) 

Sheet and channel closing are both governed by the same process, creep closure, which reduces the size of cavities of thickness and channels of radius $h$ via the same equation:

$v_c(N,h) = Ah|N|^{n-1} N$

where $N$ is the effective pressure, which we can approximate as the difference between the overburden stress of the overlying ice $\phi_i$ and the  dydnamic hydropotential of the flow $\phi$. $n$ is the exponent in Glen's flow law, which we assume to be $3$.  

The mechanisms that govern the opening of channels and sheets are different. Distributed connected networks of cavities open via ice sliding over a bumpy bed:

$v_o(u_b,h) = \frac{u_b}{l_r} (h_r - h)$

where $u_b$ is the basal slip velocity (one of the mechanims by which ice dynamics feed back with the hydrology model), $l_r$ is the length scale of bed roughness, $h_r$ is the amplitude of bed roughness, and $h$ is the local thickness of the distributed cavity.

Channel opening is controlled by the exchange of potential energy in the turbulent flow with the icy side walls. The effects of Pressure melting can also lead to channel opening; however, the same pressure melting mechanism can also restrict channel formation, especially on retrograde bed slopes. The expressions for the channel opening (and occasional closing due to pressure melting) are:





# Synthetic Greenland land terminating outlet glacier
First we need to call the mesh objects created for our shmip experiments and real glacier geometries. For more information on these experiments see the inputs directory and the mesh generation notebook.

In [2]:
def surface(x,y):
    return 6*(firedrake.sqrt(x+5000.)-firedrake.sqrt(5000.))+firedrake.Constant(1.)
def bed(x,y):
    return firedrake.Constant(0.0)

In [3]:
########### Domain Geometry SHMIP part 1: Greenland ############

Lx = 100e3
nx =50000
flux_condition = 0.0
mesh = firedrake.IntervalMesh(nx,Lx)
degree = 1
V_cg = firedrake.FunctionSpace(mesh,"CG",degree)

In [4]:
Surface=firedrake.Function(V_cg)
Bed=firedrake.Function(V_cg)
H=firedrake.Function(V_cg)
x = firedrake.SpatialCoordinate(mesh)
Surface=firedrake.interpolate(surface(x[0],0.0),V_cg)
Bed=firedrake.interpolate(bed(x[0],0.0),V_cg)
H = firedrake.interpolate(Surface-Bed,V_cg)
width = firedrake.interpolate(firedrake.Constant(20000.0),V_cg)
ub = firedrake.Function(V_cg)

#slip velocity and melt rate defined in the SHMIP experiment.
ub=firedrake.interpolate(firedrake.Constant(1e-6),V_cg)
m = firedrake.interpolate((firedrake.Constant(7.93e-9)),V_cg)
plot(H)

<IPython.core.display.Javascript object>

# Important input variables for SHMIP
The variable $m$ represents the combined fluxes associated with melt and any point sources in the mesh. This would include moulins (relavent for capturing feedbacks with surface melt on Greenland outlet glaciers and mountain glaciers) and the drainage of subglacial lakes (relavent for the Greenland near interior and Antarctic hydrology systems).

In [5]:
########### Model Initializtion ############

h_init = firedrake.Function(V_cg)
h_init = firedrake.interpolate(Constant(1.0e-9),V_cg)
S_init = firedrake.Function(V_cg)

In [6]:
# Load potential at 0 pressure
phi_m = firedrake.Function(V_cg)
phi_m = Bed*pcs['g']*pcs['rho_water']
# Ice overburden pressure
p_i = firedrake.Function(V_cg)
p_i = H*pcs['g']*pcs['rho_ice']+firedrake.Constant(1.0)
# Enforce 0 pressure bc at margin
bc = firedrake.DirichletBC(V_cg, firedrake.Constant(0.0),1)

# Initialize hydropotential
phi_init=firedrake.Function(V_cg)
phi_init=p_i + phi_m

# change constants
pcs["e_v"]=1.0e-5
pcs["h_r"]=.5

In [7]:
model_inputs = {}
model_inputs['phi_m'] = phi_m
model_inputs['p_i'] = p_i
model_inputs['phi_0'] = phi_m + p_i
model_inputs['mesh'] = mesh
model_inputs['H'] = H
model_inputs['B'] = Bed
model_inputs['u_b'] = ub
model_inputs['m'] = m
model_inputs['h_init'] = h_init
model_inputs['S_init'] = S_init
model_inputs['phi_init'] = phi_init
model_inputs['phi']=phi_init
model_inputs['d_bcs'] = [bc]
model_inputs['width'] = width
model_inputs['out_dir'] = "outputs/"
model_inputs['constants'] = pcs

In [8]:
# Create the Glads model
model = Glads1DModel(model_inputs)
# End time
T = 15.0*pcs['spd']
# Time step
dt = 60.0*30

# Geometries and inputs
Before we identify any steady state behavior in the model, we first plot the geometries of our synthetic glacier with the initial condition for channel and sheet geometry, and the melt water input that will load the system for the duration of the simulation.

In [9]:
############# Run simulation ################

while model.t < T:
    model.step(dt)
    print(model.t)
    if (int(model.t) % int(pcs['spd']) ==0):
        plot(model.N)

1800.0
3600.0
5400.0
7200.0
9000.0
10800.0
12600.0
14400.0
16200.0


ConvergenceError: Nonlinear solve failed to converge after 50 nonlinear iterations.
Reason:
   DIVERGED_MAX_IT

In [ ]:
plot(model.N)

In [ ]:
plot(model.pfo)


In [ ]:
plot(model.N)

In [ ]:
plot(model.h)

In [ ]:
phi_ds=firedrake.interpolate(-model.phi.dx(0),model.V_cg)

In [ ]:
plot(phi_ds)